In [1]:
# load libraries
import pandas as pd
import plotly.express as px
import plotly.offline as pyo
import re
import numpy as np

In [2]:
# load data
directory = "C:\\Users\\joshg\\OneDrive\\Desktop\\code_pudding\\code_pudding_v2\\code_pudding2"
df = pd.read_csv(directory + "\speed_dating_data.csv", encoding='latin1')

In [3]:
# viewing a sample of the data
print(df.info())
df.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Columns: 195 entries, iid to amb5_3
dtypes: float64(174), int64(13), object(8)
memory usage: 12.5+ MB
None


,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
7990,535,5.0,1,10,2,21,22,1,18.0,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5859,385,7.0,0,13,2,15,19,12,12.0,19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120,13,3.0,1,6,1,1,10,7,NaN,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5677,375,15.0,1,30,2,14,20,19,19.0,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4980,336,5.0,1,10,1,13,10,2,2.0,6,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
5544,369,9.0,1,18,2,14,20,5,5.0,7,...,6.0,9.0,10.0,9.0,7.0,6.0,9.0,10.0,8.0,7.0
4928,331,10.0,0,19,1,13,9,8,8.0,6,...,7.0,6.0,6.0,8.0,5.0,6.0,6.0,6.0,7.0,4.0
7727,523,15.0,0,29,2,21,22,5,5.0,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7341,498,3.0,0,5,1,20,7,5,5.0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6388,413,17.0,1,34,2,15,18,3,3.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# get rid of a bad entry
df = df.dropna(subset='pid')

## Feature Engineering

In [5]:
# number of dates per attendee
dates_count = df.groupby('iid')['iid'].count()

def get_dates_count(obs):
    return dates_count[obs['iid']]

df['dates_count'] = df.apply(get_dates_count, axis=1)

In [6]:
# check to see if above matches 'round'
df['sanity'] = df['dates_count'] - df['round']
df[df['sanity'] != 0][['iid', 'wave', 'pid', 'round', 'dates_count', 'sanity']]

,iid,wave,pid,round,dates_count,sanity
1746,122,5,112.0,10,9,-1
1747,122,5,113.0,10,9,-1
1748,122,5,114.0,10,9,-1
1749,122,5,115.0,10,9,-1
1750,122,5,116.0,10,9,-1
...,...,...,...,...,...,...
6799,453,17,435.0,11,10,-1
6800,453,17,436.0,11,10,-1
6801,453,17,437.0,11,10,-1
6802,453,17,438.0,11,10,-1


In [7]:
matches_given = df.groupby('iid')['dec'].sum()

def get_matches_given(obs):
    return matches_given[obs['iid']]

df['matches_given'] = df.apply(get_matches_given, axis=1)

In [8]:
matches = {}

for iid in df['iid'].unique():
    matches[iid] = []

def get_matches(obs):
    if obs['dec'] == 1:
        matches[obs['iid']].append(obs['pid'])
        print(obs['iid'],matches[obs['iid']], matches[obs['pid']])

df.apply(get_matches, axis=1)

matches

1 [11.0] []
1 [11.0, 12.0] []
1 [11.0, 12.0, 13.0] []
1 [11.0, 12.0, 13.0, 14.0] []
1 [11.0, 12.0, 13.0, 14.0, 15.0] []
1 [11.0, 12.0, 13.0, 14.0, 15.0, 17.0] []
1 [11.0, 12.0, 13.0, 14.0, 15.0, 17.0, 19.0] []
1 [11.0, 12.0, 13.0, 14.0, 15.0, 17.0, 19.0, 20.0] []
2 [14.0] []
2 [14.0, 17.0] []
2 [14.0, 17.0, 19.0] []
2 [14.0, 17.0, 19.0, 20.0] []
4 [14.0] []
4 [14.0, 19.0] []
4 [14.0, 19.0, 20.0] []
5 [14.0] []
5 [14.0, 16.0] []
5 [14.0, 16.0, 17.0] []
5 [14.0, 16.0, 17.0, 18.0] []
5 [14.0, 16.0, 17.0, 18.0, 19.0] []
5 [14.0, 16.0, 17.0, 18.0, 19.0, 20.0] []
6 [11.0] []
6 [11.0, 12.0] []
6 [11.0, 12.0, 14.0] []
6 [11.0, 12.0, 14.0, 17.0] []
6 [11.0, 12.0, 14.0, 17.0, 19.0] []
6 [11.0, 12.0, 14.0, 17.0, 19.0, 20.0] []
7 [14.0] []
7 [14.0, 17.0] []
7 [14.0, 17.0, 19.0] []
7 [14.0, 17.0, 19.0, 20.0] []
8 [11.0] []
8 [11.0, 12.0] []
8 [11.0, 12.0, 13.0] []
8 [11.0, 12.0, 13.0, 14.0] []
8 [11.0, 12.0, 13.0, 14.0, 15.0] []
8 [11.0, 12.0, 13.0, 14.0, 15.0, 16.0] []
8 [11.0, 12.0, 13.0, 14.0, 1

{1: [11.0, 12.0, 13.0, 14.0, 15.0, 17.0, 19.0, 20.0],
 2: [14.0, 17.0, 19.0, 20.0],
 3: [],
 4: [14.0, 19.0, 20.0],
 5: [14.0, 16.0, 17.0, 18.0, 19.0, 20.0],
 6: [11.0, 12.0, 14.0, 17.0, 19.0, 20.0],
 7: [14.0, 17.0, 19.0, 20.0],
 8: [11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0],
 9: [11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0],
 10: [13.0, 17.0, 19.0, 20.0],
 11: [],
 12: [7.0, 8.0, 9.0],
 13: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0],
 14: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0],
 15: [1.0, 2.0, 3.0, 4.0, 6.0, 7.0, 8.0, 9.0, 10.0],
 16: [1.0, 2.0, 3.0, 4.0, 6.0, 7.0, 8.0, 9.0],
 17: [4.0, 9.0],
 18: [2.0, 7.0, 8.0],
 19: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0],
 20: [8.0],
 21: [42.0, 49.0],
 22: [44.0, 45.0, 46.0, 48.0, 50.0, 53.0, 55.0],
 23: [40.0, 42.0, 43.0, 44.0, 45.0, 46.0, 49.0, 50.0],
 24: [46.0, 53.0],
 25: [],
 26: [],
 27: [46.0, 50.0],
 28: [46.0, 51.0, 53.0],
 29: [44.0, 45.0, 46.0, 48.0, 50.0, 51.0, 52.0,

In [9]:
def get_dec_o2(obs):
    if obs['iid'] in matches[obs['pid']]:
        return 1
    else:
        return 0

df['dec_o2'] = df.apply(get_dec_o2, axis=1)

In [10]:
matches_received = df.groupby('iid')['dec_o2'].sum()

def get_matches_received(obs):
    return matches_received[obs['iid']]

df['matches_received'] = df.apply(get_matches_received, axis=1)

In [11]:
mutual_count = df.groupby('iid')['match'].sum()

def get_mutual_count(obs):
    return mutual_count[obs['iid']]
    
df['mutual_count'] = df.apply(get_mutual_count, axis=1)

In [12]:
df['match_given_rate'] = df['matches_given'] / df['dates_count']
df['match_receive_rate'] = df['matches_received'] / df['dates_count']
df['mutual_rate'] = df['mutual_count'] / df['matches_given']

In [13]:
def decode_genders(obs):
    if obs['gender'] == 1:
        return 'Male'
    else:
        return 'Female'
    
df['gender_name'] = df.apply(decode_genders, axis=1)

In [14]:
goals = {1.0: 'Seemed like a fun night out',
         2.0: 'To meet new people',
         3.0: 'To get a date',
         4.0: 'Looking for a serious relationship',
         5.0: 'To say I did it',
         6.0: 'Other'}

def decode_goal(obs):
    if obs['goal'] in list(goals.keys()):
        return goals[obs['goal']]
    
df['goal_dc'] = df.apply(decode_goal, axis=1)

In [15]:
career_codes = {1.0: 'Lawyer',
                2.0: 'Academic/Research',
                3.0: 'Psychologist',
                4.0: 'Doctor/Medicine',
                5.0: 'Engineer',
                6.0: 'Creative Arts/Entertainment',
                7.0: 'Banking/Consulting/Finance/Marketing/Business/CEO/Entrepreneur/Admin',
                8.0: 'Real Estate',
                9.0: 'International/Humanitarian Affairs',
                10.0: 'Undecided',
                11.0: 'Social Work',
                12.0: 'Speech Pathology',
                13.0: 'Politics',
                14.0: 'Pro sports/Athletics',
                15.0: 'Other',
                16.0: 'Journalism',
                17.0: 'Architecture'}

career_codes = pd.Series(career_codes)

def decode_careers(obs):
    if obs['career_c'] in career_codes.keys():
        return career_codes[obs['career_c']]
    else:
        return 'Other'

df['career_name'] = df.apply(decode_careers, axis=1)

In [16]:
def demand_satisfaction(obs):
    satis = obs['satis_2']
    if satis <= 2:
        return 'Dissatisfied'
    elif 2 < satis <=4:
        return 'Somewhat Dissatisfied'
    elif 4 < satis <= 6:
        return 'Somewhat Satisfied'
    elif 6 < satis <= 8:
        return 'Satisfied'
    elif 8 < satis:
        return 'Very Satisfied'

df['satisfaction'] = df.apply(demand_satisfaction, axis=1)

In [17]:
def clean_met(obs):
    if obs['met'] == 1:
        return 'Familiar'
    else:
        return 'Strangers'
    
def clean_met_o(obs):
    if obs['met_o'] == 1:
        return 'Familiar'
    else:
        return 'Strangers'

df['met'] = df.apply(clean_met, axis=1)
df['met_o'] = df.apply(clean_met_o, axis=1)

In [18]:
length_codes = {1.0: 'Too short', 2.0: 'Too long', 3.0: 'Just right'}

def decode_length(obs):
    if obs['length'] in list(length_codes.keys()):
        return length_codes[obs['length']]
    
df['length_dc'] = df.apply(decode_length, axis=1)

In [19]:
small_waves = [6,16,18,20]
medium_waves = [1,3,5,7,9,11,12,15]
large_waves = [2,4,6,8,10,13,14,16]

def get_wave_size(obs):
    if obs['wave'] in small_waves:
        return 'small'
    elif obs['wave'] in medium_waves:
        return 'medium'
    elif obs['wave'] in large_waves:
        return 'large'
    
df['wave_size'] = df.apply(get_wave_size, axis=1)

In [20]:
numdat_codes = {1.0: 'Too few', 2.0: 'Too many', 3.0: 'Just right'}

def decode_numdat(obs):
    if obs['numdat_2'] in list(length_codes.keys()):
        return numdat_codes[obs['numdat_2']]
    
df['numdat_dc'] = df.apply(decode_numdat, axis=1)

In [21]:
def get_age_group(obs):
    if obs['age'] < 24:
        return '23 and under'
    elif 24 <= obs['age'] <= 28:
        return '24 to 28'
    elif 28 < obs['age']:
        return '29 and over'
    
df['age_group'] = df.apply(get_age_group, axis=1)

In [22]:
def get_contacted(obs):
    if obs['you_call'] >= 1:
        return True
    elif obs['them_cal'] >= 1:
        return True
    else:
        return False
    
df['contacted'] = df.apply(get_contacted, axis=1)

In [ ]:
pref_alloc = ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']
waves_to_scale = [6, 7, 8, 9]
scaled_atbs = {}

for iid in df['iid'].unique():
    scaled_atbs[iid] = {}

def scale_prefs(obs):
    if obs['wave'] in waves_to_scale:
        sum_pref = obs[pref_alloc].sum()
        for atb in pref_alloc:
            try:
                scaled_atbs[obs['iid']][atb] = obs[atb] / sum_pref * 100
            except:
                scaled_atbs[obs['iid']][atb] = np.nan

iids_to_scale = df.query('wave in @waves_to_scale')['iid']

df.apply(scale_prefs, axis=1)

for iid in iids_to_scale:
    print(iid, scaled_atbs[iid])

df.replace(scaled_atbs)

In [ ]:
df_uniq_id = df.drop_duplicates('iid')
df_uniq_id

In [ ]:
career_counts = df_uniq_id[['career_name']].value_counts().reset_index()
career_counts.columns = ['career_name', 'count']

#career_counts

In [ ]:
career_success = df_uniq_id.groupby('career_name')['match_receive_rate'].mean().sort_values(ascending=False).reset_index()
career_success.columns = ['career_name', 'match_receive_rate']
#career_success[['career_name', 'match_receive_rate']]

In [ ]:
age_count = df_uniq_id.groupby(['gender', 'age'])['match_receive_rate'].count()
#age_count

In [ ]:
age_success = df_uniq_id.groupby('age')['match_receive_rate'].mean()
#age_success

In [ ]:
scorecard_ratings = ['attr', 'sinc', 'intel', 'fun', 'amb']
others_ratings = ['attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o']
self_rate_before = ['attr3_1', 'sinc3_1', 'intel3_1', 'fun3_1', 'amb3_1']
self_rate_day_after = ['attr3_2', 'sinc3_2', 'intel3_2', 'fun3_2', 'amb3_2']
self_rate_weeks_after = ['attr3_3', 'sinc3_3', 'intel3_3', 'fun3_3', 'amb3_3']
self_rate_before = ['attr3_1', 'sinc3_1', 'intel3_1', 'fun3_1', 'amb3_1']
others_ratings_avg = df.groupby('pid')[['attr', 'sinc', 'intel', 'fun', 'amb']].mean()


In [ ]:
self_ratings = df[['iid','attr3_2', 'sinc3_2', 'intel3_2', 'fun3_2', 'amb3_2']]
self_ratings = self_ratings.drop_duplicates(subset='iid')

all_ratings = self_ratings.merge(others_ratings_avg, left_on='iid', right_on='pid')

all_ratings['diff_attr'] = all_ratings['attr'] - all_ratings['attr3_2']
all_ratings['diff_sinc'] = all_ratings['sinc'] - all_ratings['sinc3_2']
all_ratings['diff_intel'] = all_ratings['intel'] - all_ratings['intel3_2']
all_ratings['diff_fun'] = all_ratings['fun'] - all_ratings['fun3_2']
all_ratings['diff_amb'] = all_ratings['amb'] - all_ratings['amb3_2']

all_ratings.loc[:,'diff_attr':].describe()

## Visualizations

In [ ]:
# histogram of frequency of input column
inputs = ['age','career_name','goal']

for input in inputs:
    fig = px.histogram(df_uniq_id, x=input, color='gender', barmode='overlay')
    fig.show()
    #pyo.plot(fig, filename=input + '_histogram.html')

In [ ]:
# pie chart of attendee responses per input
inputs = ['satisfaction', 'length', 'numdat_2']

for input in inputs:
    breakdown_by_input = df_uniq_id.groupby(input)['iid'].count().reset_index()
    fig = px.pie(breakdown_by_input, values='iid', names=input)
    fig.show()
    #pyo.plot(fig, filename=input + '_pie.html')

In [ ]:
# bar chart of match receive rate by input
inputs = ['age', 'career_name', 'met_o', 'goal']

for input in inputs:
    match_rate_by_input = df_uniq_id.groupby(['gender', input])['match_receive_rate'].mean().reset_index()

    fig = px.bar(match_rate_by_input, x=input, y='match_receive_rate', color='gender', color_discrete_sequence=['#FF5733', '#33FF7A'], barmode='overlay')
    fig.show()
    #pyo.plot(fig, filename=input + '_match_bar.html')

In [ ]:
rating1 = self_rate_day_after
rating2 = self_rate_day_after
rating_diff = {}

for atb in range(0,len(rating1)):
    temp = re.compile("([a-zA-Z]+)([0-9]+)")
    atb_clean = temp.match(rating1[atb]).groups()[0]
    rating_diff[atb_clean + '_diff'] = df[rating2[atb]] - df[rating1[atb]] 

def get_rating_diff(obs):
    for rating in list(rating_diff.keys()):
        print(rating_diff[rating][obs])

#for i in range(0, len(rating_diff)):
#    df[list(rating_diff.keys())[i]] = df.apply(get_rating_diff, axis=1)

'''df.apply(get_rating_diff, axis=1)

df_uniq_id_temp = df.groupby('iid')[list(rating_diff.keys())].mean()

df_uniq_id_temp'''

In [ ]:
corr_cols = ['shar_o']
for i in range(0,len(others_ratings)):
    corr_cols.append(others_ratings[i])
target = 'dec_o2'
corr1 = df[corr_cols].corrwith(df[target])

corr_cols2 = ['shar']
for i in range(0,len(scorecard_ratings)):
    corr_cols2.append(scorecard_ratings[i])
target2 = 'dec'
corr2 = df[corr_cols2].corrwith(df[target2])

print("Correlations between others' ratings of attendee's attributes and whether or not the other says 'match'\n", corr1)
print("Correlations between attendee's ratings of others' attributes and whether or not they say 'match'\n", corr2)


In [ ]:
# attendees per career
df_uniq_id['career_name'].value_counts()

In [ ]:
# attendees by age
df_uniq_id['age'].value_counts()

## Creating and Exporting Data for Website Display

In [ ]:
orientation = 'records'

### Organizers

#### How do people feel about 4-minutes as a length for each date?
horizontal bar

In [ ]:
# get value counts for each value in from df_uniq_id
date_length_survey = df_uniq_id['length_dc'].value_counts().reset_index()
date_length_survey = date_length_survey.rename(
    columns={'index': 'response', 'length_dc': 'amount'})
date_length_survey.to_json('./date_length_survey.json', orient=orientation)

date_length_survey


In [ ]:

px.bar(date_length_survey, x='amount', y='response', orientation='h')

#### How many dates should each person go on?
treeplot


In [ ]:
# count values in numdat_2 and group by wave size
num_dates_survey_small_wave = df_uniq_id[df_uniq_id['wave_size'] == 'small']['numdat_dc'].value_counts().reset_index()
num_dates_survey_small_wave = num_dates_survey_small_wave.rename(
    columns={'index':'response', 'numdat_dc':'amount'})
num_dates_survey_small_wave.to_json('./num_dates_survey_small_wave.json', orient=orientation)

num_dates_survey_medium_wave = df_uniq_id[df_uniq_id['wave_size'] == 'medium']['numdat_dc'].value_counts().reset_index()
num_dates_survey_medium_wave = num_dates_survey_medium_wave.rename(
    columns={'index':'response', 'numdat_dc':'amount'})
num_dates_survey_medium_wave.to_json('./num_dates_survey_medium_wave.json', orient=orientation)

num_dates_survey_large_wave = df_uniq_id[df_uniq_id['wave_size'] == 'large']['numdat_dc'].value_counts().reset_index()
num_dates_survey_large_wave = num_dates_survey_large_wave.rename(
    columns={'index':'response', 'numdat_dc':'amount'})
num_dates_survey_large_wave.to_json('./num_dates_survey_large_wave.json', orient=orientation)

print('small waves\n', num_dates_survey_small_wave.values)
print('medium waves\n', num_dates_survey_medium_wave.values)
print('large waves\n', num_dates_survey_large_wave.values)

#### What professions get the most matches from their dates?
lollipop

In [ ]:
# get a sorted list of mean match_receive_rate by career
career_match_success = pd.Series(df_uniq_id.groupby('career_name')['match_receive_rate'].mean().sort_values(ascending=False)).reset_index()
career_match_success = career_match_success.rename(
    columns={'index':'career', 'match_receive_rate':'like_receive_rate'})
career_match_success.to_json('./career_match_success.json', orient=orientation)

career_match_success

#### Are participants more likely to receive a match if they've met before?
bar

In [ ]:
# get mean 'match_receive_rate' by response to 'met_o'
met_before_gender_match_success = df_uniq_id.groupby(['gender_name', 'met_o'])['match_receive_rate'].mean().sort_values(ascending=False).reset_index()
met_before_gender_match_success = met_before_gender_match_success.rename(
    columns={'gender_name':'gender', 'met_o':'status', 'match_receive_rate':'like_receive_rate'})
met_before_gender_match_success.to_json('./met_before_gender_match_success.json', orient=orientation)

met_before_match_success = df_uniq_id.groupby(['met_o'])['match_receive_rate'].mean().sort_values(ascending=False).reset_index()
met_before_match_success = met_before_match_success.rename(
    columns={'met_o':'status', 'match_receive_rate':'like_receive_rate'})
met_before_match_success.to_json('./met_before_match_success.json', orient=orientation)

print('with genders\n', met_before_gender_match_success)
print('altogether\n', met_before_match_success)


#### What ages get the most matches?
bar

In [ ]:
# get a sorted list of mean match_receive_rate by age
age_gender_match_success = df_uniq_id.groupby(['gender_name', 'age'])['match_receive_rate'].mean().sort_values(ascending=False).reset_index()
age_gender_match_success = age_gender_match_success.rename(
    columns={'gender_name':'gender', 'age':'age', 'match_receive_rate':'like_receive_rate'})
age_gender_match_success.to_json('./age_gender_match_success.json', orient=orientation)

age_match_success = df_uniq_id.groupby(['age'])['match_receive_rate'].mean().sort_values(ascending=False).reset_index()
age_match_success = age_match_success.rename(
    columns={'age':'age', 'match_receive_rate':'like_receive_rate'})
age_match_success.to_json('./age_match_success.json', orient=orientation)

age_group_gender_match_success = df_uniq_id.groupby(['gender_name', 'age_group'])['match_receive_rate'].mean().sort_values(ascending=False).reset_index()
age_group_gender_match_success = age_group_gender_match_success.rename(
    columns={'gender_name':'gender', 'match_receive_rate':'like_receive_rate'})
age_group_gender_match_success.to_json('./age_group_gender_match_success.json', orient=orientation)

age_group_match_success = df_uniq_id.groupby(['age_group'])['match_receive_rate'].mean().sort_values(ascending=False).reset_index()
age_group_match_success = age_group_match_success.rename(
    columns={'match_receive_rate':'like_receive_rate'})
age_group_match_success.to_json('./age_group_match_success.json', orient=orientation)

print('gendered, by age\n', age_gender_match_success)
print('altogether, by age\n', age_match_success)
print('gendered, by age group\n', age_group_gender_match_success)
print('altogether, by age group\n', age_group_match_success)

#### How satisfied were participants with the people they met?
circle-packing

In [ ]:
# count values in satisfaction
satisfaction_survey = df_uniq_id['satisfaction'].value_counts().reset_index()
satisfaction_survey = satisfaction_survey.rename(
    columns={'index': 'response', 'satisfaction': 'amount'})
satisfaction_survey.to_json('./satisfaction_survey.json', orient=orientation)

satisfaction_survey

### Participants


#### What hobbies are most popular among participants?
word cloud


In [ ]:
# get mean hobby ratings and sort
hobby_ratings = df_uniq_id.loc[:,'sports':'yoga'].mean().sort_values(ascending=False).reset_index()
hobby_ratings = hobby_ratings.rename(
    columns={'index': 'hobby', 0: 'avg_rating'})
hobby_ratings.to_json('./hobby_ratings.json', orient=orientation)

hobby_ratings

#### What is the age of participants?
pie or donut?

In [ ]:
age_counts = df_uniq_id['age'].value_counts().reset_index()
age_counts = age_counts.rename(
    columns={'index': 'age', 'age': 'amount'})
age_counts.to_json('./age_counts.json', orient=orientation)


age_group_counts = df_uniq_id['age_group'].value_counts().reset_index()
age_group_counts = age_group_counts.rename(
    columns={'index': 'age_group', 'age_group': 'amount'})
age_group_counts.to_json('./age_group_counts.json', orient=orientation)

print('counts by age\n', age_counts)
print('counts by age group\n', age_group_counts)

#### What are participants goals?


In [ ]:
# count values in goal_dc
goals_survey = df_uniq_id['goal_dc'].value_counts().reset_index()
goals_survey = goals_survey.rename(
    columns={'index': 'goal', 'goal_dc': 'amount'})
goals_survey.to_json('./goals_survey.json', orient=orientation)

goals_survey

#### How likely are you to match? 
funnel/sankey


In [ ]:
# get mean of matches logged, mutual matches, a "yes or no" to calls either way, and date
match_to_date_funnel = pd.Series({'Likes': df_uniq_id['matches_given'].mean(),
                                  'Matches': df_uniq_id['mutual_count'].mean(),
                                  'Contact made': df_uniq_id['contacted'].mean(),
                                  'Had a date': df_uniq_id['date_3'].fillna(0).mean()}).reset_index()

match_to_date_funnel = match_to_date_funnel.rename(
    columns={'index': 'stage', 0: 'avg_per_participant'})
match_to_date_funnel.to_json('./match_to_date_funnel.json', orient=orientation)

match_to_date_funnel

#### Does self-perception match dates' ratings?
scatter

In [ ]:
traits = ['Attractiveness', 'Sincerity', 'Intelligence', 'Fun', 'Ambition']

traits_dict = {0: 'Female',
               1: 'Male'}

for i in range(0,len(others_ratings)):
    traits_dict[others_ratings[i]] = traits[i] + '_others'
    traits_dict[self_rate_before[i]] = traits[i] +'_self'

self_vs_others_ratings = df[list(others_ratings) + list(self_rate_before)].mean().reset_index()
self_vs_others_ratings = self_vs_others_ratings.rename(
    columns={'index': 'trait', 0: 'avg_rating'}).replace(traits_dict)
self_vs_others_ratings.to_json('./self_vs_others_ratings.json', orient=orientation)

self_vs_others_gender_ratings = df.groupby('gender')[list(others_ratings) + list(self_rate_before)].mean().reset_index()
self_vs_others_gender_ratings = self_vs_others_gender_ratings.rename(
    columns=traits_dict).replace(traits_dict)
self_vs_others_gender_ratings.to_json('./self_vs_others_gender_ratings.json', orient=orientation)

print('altogether\n', self_vs_others_ratings)
print('gendered\n', self_vs_others_gender_ratings)

#### How important are these traits?
lollipop


In [ ]:
# get mean of each attributes preference
traits.append('Shared Interests')

trait_importance = df[pref_alloc].mean()
trait_importance = trait_importance.set_axis(['Attractiveness', 'Sincerity', 'Intelligence', 'Fun', 'Ambition', 'Shared Interests']).reset_index()
trait_importance = trait_importance.rename(
    columns={'index': 'trait', 0: 'score'})
trait_importance.to_json('./trait_importance.json', orient=orientation)

trait_importance

#### Which traits correlate best with receiving a like?
scatter

In [ ]:
# get mean attribute scores from others compared to match_receive_rate
trait_match_correlation = others_ratings_avg.merge(df_uniq_id[['iid','match_receive_rate']], left_on='pid', right_on='iid').drop('iid', axis=1)
trait_match_correlation

for trait in list(others_ratings_avg.columns):
    trait_match_correlation[[trait, 'match_receive_rate']].to_json('./' + trait + '_correlation.json', orient=orientation)